In [1]:
%pylab inline
import pandas

/home/ubuntu/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [156]:
TRAINFILE='/home/ubuntu/data/training_input.csv'
LABELFILE='/home/ubuntu/data/challenge_output_data_training_file_prediction_of_trading_activity_within_the_order_book.csv'
CHUNKSIZE=8
NCHUNKS=10
LASTROW=7
DAY=4679
USEFEATURES = ['ID', 'bid_1', 'ask_1', 'bid_size_1', 'ask_size_1', 'nb_trade', 'bid_entropy_1', 'ask_entropy_1', 'bid_size_2', 'ask_size_2']
NEWFEATURES = ['bid_1', 'spread', 'bid_plus_ask', 'bid_pct', 'nb_trade']+['bid_size_2', 'ask_size_2']
TRAINPICKLE='train.pkl'

In [157]:
train_all = pandas.read_csv(TRAINFILE, usecols=USEFEATURES)

In [158]:
train_all['spread'] = train_all['ask_1']-train_all['bid_1']
train_all['bid_plus_ask'] = train_all['bid_size_1']-train_all['ask_size_1']
train_all['bid_pct'] = train_all['bid_size_1']/train_all['bid_plus_ask']

In [159]:
train_diff = train_all[NEWFEATURES].diff()

In [160]:
past_diff_feat = np.hstack([train_diff.iloc[i::8, :].values for i in range(1,8)])

In [175]:
fut_diff_feat = np.hstack([train_diff.iloc[i+7::8, :].values for i in range(1,8)])
# fix end
fut_diff_feat = np.r_[fut_diff_feat, np.zeros((1, fut_diff_feat.shape[1]))]

In [176]:
pres_orig_feat = train_all.iloc[7::8][NEWFEATURES].values

In [177]:
past_orig_feat = np.hstack([train_all.iloc[i::8, 1:8].values for i in range(8)])

In [178]:
time_feat = (train_all.iloc[7::8, 0].values-1)%DAY

In [179]:
entropy_feat = train_all.iloc[7::8, -2:]

In [180]:
#train=np.c_[past_diff_feat, pres_orig_feat, time_feat]
#train=np.c_[pres_orig_feat, time_feat]
#train=pres_orig_feat
train=np.c_[past_diff_feat, fut_diff_feat, pres_orig_feat, time_feat]

In [181]:
np.save(TRAINPICKLE, train, allow_pickle=True)

In [182]:
label = pandas.read_csv(LABELFILE, sep=';')['TARGET'].values

In [183]:
import xgboost as xgb

In [185]:
dtrain = xgb.DMatrix(data=np.load(TRAINPICKLE+'.npy'), label = label)
prior=label.mean()
params={}
params['bst:eta'] = 0.1
params['bst:max_depth'] = 10
params['min_child_weight'] = 4
params['objective'] = 'binary:logistic'
params['nthread'] = 4
params['eval_metric'] = 'error'
params['learning_rate'] = 0.1
params['lambda'] = 0.1
params['base_score'] = prior
res = xgb.cv(params, dtrain, num_boost_round=1000, verbose_eval=True, nfold=4, seed=0, early_stopping_rounds=10, show_stdv=True)

[0]	train-error:0.338193+0.000161631	test-error:0.338193+0.000485133
[1]	train-error:0.338193+0.000161631	test-error:0.338193+0.000485133
[2]	train-error:0.337298+0.000207679	test-error:0.337377+0.000646512
[3]	train-error:0.331752+0.000398358	test-error:0.331793+0.000599183
[4]	train-error:0.326623+0.000365246	test-error:0.327093+9.06725e-05
[5]	train-error:0.320452+0.000321476	test-error:0.320962+0.00059469
[6]	train-error:0.313783+0.000368305	test-error:0.314622+0.0010655
[7]	train-error:0.307842+0.000422803	test-error:0.309049+0.00055372
[8]	train-error:0.302484+0.000659994	test-error:0.303938+0.000684276
[9]	train-error:0.298766+0.000286069	test-error:0.300253+0.000606538
[10]	train-error:0.296259+0.000191385	test-error:0.297639+0.000709502
[11]	train-error:0.294462+0.00030653	test-error:0.295796+0.000618134
[12]	train-error:0.293124+0.00033066	test-error:0.294533+0.0006846
[13]	train-error:0.291791+0.00043421	test-error:0.293543+0.000556602
[14]	train-error:0.290921+0.000223608	t